In [ ]:
# imports
import pandas as pd
from pandas.io.json import json_normalize
import requests
import os
import json
import sqlite3
from sqlalchemy import create_engine

# Foursquare

Send a request to Foursquare with a small radius (1000m) with the location of your choice

In [ ]:
foursquare_key = os.environ["FOURSQUARE_API_KEY"]
lat = '49.26660189762465'
long = '-123.25003857290854'

headers = {'Accept': 'application/json'}
headers['Authorization'] = foursquare_key

url_append = f'?ll={lat}%2C{long}&radius=5000&categories=13000'

url = "https://api.foursquare.com/v3/places/search"

In [ ]:
def fsq_get_venues(lat, long):
    """
    1. Executes get request from FSQ API
    2. Prints status code of get request
    3. Returns normalized DataFrame from JSON file retrieved by the get request
    """
    res = requests.get(url + url_append, headers = headers)
    print('The status code is: ' + str(res.status_code))
    request_json = res.json()

    fsq_data = pd.json_normalize(request_json, record_path = 'results')
    return fsq_data

In [ ]:
df = fsq_get_venues(lat, long)

Parse through the response to get the POI details you want (rating, name, location, etc)

In [ ]:
df[['name', 'categories', 'location.address']]

Put your parsed results into a DataFrame

In [ ]:
parsed_df = df[['name', 'categories', 'location.address']]
parsed_df

In [ ]:
# Creating database and inserting fsq table

with sqlite3.connect('mini_project_ii.db') as con:
    cur = con.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS fsq_table (id INTEGER PRIMARY KEY AUTOINCREMENT,name TEXT NOT NULL,type TEXT,address TEXT);")
    cur.execute("INSERT INTO fsq_table (name, type, address) VALUES ('Loafe Cafe', 'Café', '6163 University Blvd');")
    cur.execute("INSERT INTO fsq_table (name, type, address) VALUES ('Mercante', 'Pizzeria', '6488 University Blvd');")
    cur.execute("INSERT INTO fsq_table (name, type, address) VALUES ('Great Dane Coffee', 'Café', '6011 Walter Gage Rd');")
    cur.execute("INSERT INTO fsq_table (name, type, address) VALUES ('The Boulevard Coffee Roasting Co', 'Coffee Shop', '5970 University Blvd');")
    cur.execute("INSERT INTO fsq_table (name, type, address) VALUES ('Koerners Pub', 'Arts and Entertainment', '6371 Crescent Rd');")
    cur.execute("INSERT INTO fsq_table (name, type, address) VALUES ('Beyond Bread Ltd', 'Bakery', '3686 4th Ave W');")
    cur.execute("INSERT INTO fsq_table (name, type, address) VALUES ('JJ Bean', 'Coffee Shop', '6005 University Blvd');")
    cur.execute("INSERT INTO fsq_table (name, type, address) VALUES ('Breka Downtown Bakery Cafe Inc', 'Bakery', '3750 4th Ave W');")
    cur.execute("INSERT INTO fsq_table (name, type, address) VALUES ('Rajio Japanese Public House', 'Japanese Restaurant', '3763W 10th Ave W');")
    cur.execute("INSERT INTO fsq_table (name, type, address) VALUES ('Biercraft Wesbrook at UBC', 'Bar', '3340 Shrum Lane');")

In [ ]:
table = 'fsq_table'

with create_engine('sqlite:///mini_project_ii.db').connect() as con:
    fsq_table_df = pd.read_sql_table(table, con)

fsq_table_df

# Yelp

Send a request with the same location paramaters (location, radius, etc)

In [ ]:
yelp_key = os.environ["YELP_API_KEY"]
lat = '49.26660189762465'
long = '-123.25003857290854'

url = "https://api.yelp.com/v3/businesses/search"
headers = {'Authorization': 'Bearer %s' % yelp_key}
params = {'latitude': lat,
          'longitude': long,
          'radius': 5000,
          'categories': 'food'}

In [ ]:
def yelp_get_venues(lat, long):
    """
    1. Executes get request from Yelp API
    2. Prints status code of get request
    3. Returns normalized DataFrame from JSON file retrieved by the get request
    """
    res = requests.get(url, params = params, headers = headers)
    print('The status code is: ' + str(res.status_code))
    request_json = res.json()

    yelp_data = pd.json_normalize(request_json, record_path = 'businesses')
    return yelp_data

In [ ]:
df = yelp_get_venues(lat, long)

Parse through your result and get POI details

In [ ]:
df[['name', 'categories', 'location.address1', 'rating']]

Put your parsed results into a DataFrame

In [ ]:
parsed_df = df[['name', 'categories', 'location.address1', 'rating']]
parsed_df

In [ ]:
# Creating database and inserting yelp table

with sqlite3.connect('mini_project_ii.db') as con:
    cur = con.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS yelp_table (id INTEGER PRIMARY KEY AUTOINCREMENT,name TEXT NOT NULL,type TEXT,address TEXT,rating DECIMAL);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Doughgirls', 'cafes', '3322 Shrum Lane', 4.5);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Grounds For Coffee', 'coffee', '2565 Alma Street', 4.0);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('The PokéMan', 'poke', '3742 10th Avenue W', 4.5);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('The Boulevard Coffee Roasting', 'coffee', '5970 University Boulevard', 3.5);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Mercante', 'pizza', '6488 University Boulevard', 3.5);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Aphrodites Organic Pie Shop', 'gluten_free', '3598 W 4th Avenue', 3.5);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Rain or Shine', 'icecream', '6001 University Boulevard', 4.0);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Caffe W', 'coffee', '3646 W Broadway', 4.5);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Bean Around The World UBC', 'coffee', '6308 Thunderbird Blvd', 4.0);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Pearl Fever Tea House', 'coffee', '2182 Western Parkway', 3.5);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Blue Chip Cookies', 'desserts', '1302-6133 Student Union Boulevard', 4.0);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('JJ Bean', 'coffeeroasteries', '6005 University Boulevard', 4.0);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Perchance', 'coffee', '3363 Dunbar St', 4.5);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Breka Bakery & Cafe', 'bakeries', '3750 W 4th Avenue', 4.0);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Beyond Bread Artisan Bakery', 'bakeries', '3686 W 4th Avenue', 4.0);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('UBC Farm Farmers Market', 'farmersmarket', '3461 Ross Drive', 4.5);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Stongs Market', 'grocery', '4221 Dunbar Street', 4.0);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Bean Around the World Coffees', 'coffee', '4456 10th Avenue W', 4.0);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Mix the Bakery', 'bakeries', '4430 10th Avenue W', 3.5);")
    cur.execute("INSERT INTO yelp_table (name, type, address, rating) VALUES ('Steves Poke Bar', 'poke', '3-5990 University Blvd', 4.5);")

In [ ]:
table = 'yelp_table'

with create_engine('sqlite:///mini_project_ii.db').connect() as con:
    yelp_table_df = pd.read_sql(table, con)

yelp_table_df

## Top 10 POIs based on rating

In [ ]:
sql = "SELECT name, rating FROM yelp_table ORDER BY rating DESC LIMIT 10"

with create_engine('sqlite:///mini_project_ii.db').connect() as con:
    top_rating_df = pd.read_sql(sql, con)

top_rating_df

# Google (stretch)

Use the same process as the first two APIs

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

Get the top 10 restaurants according to their rating

# Travelling Salesman Problem (stretch)

If you have time, follow the steps in the [ortools tutorial](https://developers.google.com/optimization/routing/tsp) using Google's [Directions API](https://developers.google.com/maps/documentation/directions/start).